# **13.4 Finally_and_Else**

The `try/except` block has two optional extra clauses that give you much finer control over what happens after error handling. The `else` clause runs only when **no exception occurred**, and the `finally` clause runs **no matter what** — success, failure, or even if an unhandled exception escapes. Together these make your Pokemon game's cleanup and success logic crystal clear.

---

## **The else Clause**

The `else` block runs only if the `try` block completed **without raising any exception**. It is the ideal place for code that should only execute on success — keeping it separate from the `try` block makes the intent obvious and prevents accidentally masking new errors.

In [ ]:
def attempt_catch(pokemon_name: str, catch_rate: int):
    """
    Attempt to catch a Pokemon.
    - try:    the risky operation (lookup + roll)
    - except: handle failures
    - else:   success-only celebration — runs only if no exception
    """
    import random

    pokedex = {"Pikachu": 190, "Mewtwo": 3, "Charizard": 45}

    try:
        rate = pokedex[pokemon_name]          # KeyError if not found
        success = random.random() < rate / 255
        if not success:
            raise RuntimeError(f"{pokemon_name} broke free!")
    except KeyError:
        print(f"  Error: '{pokemon_name}' not in Pokedex")
    except RuntimeError as e:
        print(f"  {e}")
    else:
        # Only runs if the try block raised NO exception
        print(f"  {pokemon_name} was caught! Added to your team.")
        return True

    return False

print("Throwing Pokeball at Pikachu:")
attempt_catch("Pikachu", 190)

print("\nThrowing Pokeball at Eevee:")
attempt_catch("Eevee", 45)   # Not in pokedex

---

## **Why else and Not Just More try Code?**

You might wonder: why not just put the success code at the bottom of the `try` block? The difference matters — code inside `try` is also protected, which means a new error there would be silently caught. The `else` block is **outside** that safety net, so any errors there propagate normally.

In [ ]:
pokedex = {"Pikachu": "Electric", "Charizard": "Fire"}

# WITHOUT else — both errors are masked by the same except
def lookup_bad(name):
    try:
        ptype = pokedex[name]        # KeyError we want to catch
        # Suppose there's a bug in the success code below:
        result = ptype.nonexistent() # AttributeError — ALSO caught! Hidden bug.
    except KeyError:
        print(f"  '{name}' not found")

# WITH else — only the risky operation is protected
def lookup_good(name):
    try:
        ptype = pokedex[name]        # Only this is protected
    except KeyError:
        print(f"  '{name}' not found")
    else:
        # Bug here would NOT be silently caught — it propagates clearly
        print(f"  {name} is {ptype} type")

print("Without else:")
lookup_bad("Pikachu")    # Bug hidden — you'd never know

print("\nWith else:")
lookup_good("Pikachu")   # Works correctly
lookup_good("Eevee")     # Shows 'not found' cleanly

---

## **The finally Clause**

The `finally` block runs **always** — whether the try succeeded, an exception was caught, an exception went uncaught, or even if there was a `return` statement inside `try` or `except`. It is the right place for cleanup code: closing files, releasing locks, disconnecting from databases.

In [ ]:
def save_game(trainer_name: str, data: dict):
    """
    Save game data to a file.
    finally ensures we always report the save attempt outcome.
    """
    import json
    filename = f"{trainer_name}_save.json"
    print(f"  Attempting to save to '{filename}'...")

    try:
        with open(filename, 'w') as f:
            json.dump(data, f)
        print(f"  Save succeeded")
    except OSError as e:
        print(f"  Save failed: {e}")
    finally:
        # ALWAYS runs — even if an unhandled exception escapes
        print(f"  Save attempt complete (finally)")

save_data = {"trainer": "Ash", "badges": 8, "team": ["Pikachu", "Charizard"]}

print("Normal save:")
save_game("ash", save_data)

print("\nBad filename (/ not allowed):")
save_game("/bad/path/trainer", save_data)

---

## **finally Runs Even With return**

This is a subtle but important behaviour: even if `try` or `except` has a `return` statement, `finally` still executes before the function actually returns. This makes it reliable for cleanup regardless of the exit path.

In [ ]:
def connect_to_pc_box(trainer: str):
    """
    Simulate connecting to Pokemon storage.
    finally runs even when we return early from except.
    """
    print(f"  Opening connection for {trainer}...")
    try:
        if trainer == "Unknown":
            raise ValueError("Trainer not registered")
        print(f"  Connected! Loading {trainer}'s boxes...")
        return {"box_1": ["Eevee", "Snorlax"], "box_2": ["Gengar"]}  # Returns here
    except ValueError as e:
        print(f"  Connection refused: {e}")
        return None                        # Returns here on error
    finally:
        # Runs BEFORE either return above completes
        print(f"  Closing connection (always happens)")

print("Known trainer:")
boxes = connect_to_pc_box("Ash")
print(f"  Got boxes: {boxes}\n")

print("Unknown trainer:")
boxes = connect_to_pc_box("Unknown")
print(f"  Got boxes: {boxes}")

---

## **Full try / except / else / finally**

All four clauses together give you complete control. Here is the full structure and exactly when each part runs.

In [ ]:
def battle_summary(attacker: dict, defender: dict):
    """
    Run a battle and produce a summary.

    try    — the risky battle calculation
    except — bad data (missing keys, bad types)
    else   — success-only: print the outcome
    finally — always: log that the battle was attempted
    """
    import random
    log = []

    try:
        atk_name  = attacker['name']            # KeyError if missing
        def_name  = defender['name']
        power     = int(attacker['attack'])     # ValueError if non-numeric
        damage    = power * random.randint(85, 100) // 100
        new_hp    = defender['hp'] - damage

    except KeyError as e:
        log.append(f"SKIP: missing field {e}")
    except ValueError as e:
        log.append(f"SKIP: bad stat value — {e}")

    else:
        # Only if try succeeded
        outcome = "fainted" if new_hp <= 0 else f"HP → {new_hp}"
        log.append(f"{atk_name} hit {def_name} for {damage} ({outcome})")

    finally:
        # Always
        log.append("Battle turn logged.")

    return log

pikachu  = {"name": "Pikachu",  "attack": 55, "hp": 35}
onix     = {"name": "Onix",     "attack": 45, "hp": 70}
broken   = {"name": "Broken",   "attack": "high"}

for atk, dfn in [(pikachu, onix), (broken, onix)]:
    result = battle_summary(atk, dfn)
    print("\n".join(result), "\n")

---

## **Practice Exercises**

### **Task 1: else on Success**

Add an `else` clause that prints a success message only when no exception occurs.

**Expected Output:**
```
Level parsed: 25
Success! Valid level entered.
```

In [ ]:
user_input = "25"

# Your code here (try/except/else):


### **Task 2: else Skipped on Error**

Show that `else` is skipped when an exception is raised.

**Expected Output:**
```
Error: 'Pikachu' is not numeric
(no success message printed)
```

In [ ]:
user_input = "Pikachu"

# Your code here:


### **Task 3: finally Always Runs**

Demonstrate `finally` running on both success and failure.

**Expected Output:**
```
--- Input: '25' ---
Level: 25
finally: done
--- Input: 'bad' ---
Error: bad input
finally: done
```

In [ ]:
for raw in ["25", "bad"]:
    print(f"--- Input: {raw!r} ---")
    # Your try/except/finally here:


### **Task 4: finally With return**

Show that `finally` runs even when `try` contains a `return`.

**Expected Output:**
```
finally: closing file
Result: 25
```

In [ ]:
def read_level():
    try:
        return 25   # Returns here
    finally:
        # Your code here:
        pass

result = read_level()
print(f"Result: {result}")

### **Task 5: Save with finally**

Write a save function that always prints "save attempt complete" in `finally`.

**Expected Output:**
```
Saved successfully
Save attempt complete
```

In [ ]:
# Your code here:


### **Task 6: All Four Clauses**

Write a function using `try`, `except`, `else`, and `finally` all together.

**Expected Output:**
```
Caught Pikachu!
Pokeball throw logged
```

In [ ]:
# Your code here:


### **Task 7: Count Successes**

Loop over inputs using `else` to count only the successful parses.

**Expected Output:**
```
Successful parses: 3
```

In [ ]:
inputs = ["25", "Pikachu", "36", "bad", "8"]
successes = 0

# Your code here:


### **Task 8: Resource Cleanup**

Use `finally` to always print "connection closed" when simulating a database query.

**Expected Output:**
```
Query result: Pikachu
Connection closed
```

In [ ]:
db = {"001": "Bulbasaur", "025": "Pikachu", "006": "Charizard"}

def query(pokemon_id: str):
    # Your try/except/finally here:
    pass

query("025")

### **Task 9: else vs try Body**

Show that putting extra code in `else` (instead of `try`) prevents it from being accidentally caught.

**Expected Output:**
```
Electric
```

In [ ]:
pokedex = {"Pikachu": "Electric"}

# Place the dict lookup in try, and the print in else:


### **Task 10: Battle Logger**

Write `log_battle(attacker, defender)` using all four clauses. `else` records the outcome; `finally` always appends "turn complete".

**Expected Output:**
```
['Pikachu hit Onix', 'turn complete']
['SKIP: missing attack stat', 'turn complete']
```

In [ ]:
# Your code here:


---

## **Summary**

- `else` runs only when the `try` block raised **no** exception
- Keeps success-only code clearly separated from the risky operation
- Errors inside `else` propagate normally — they are **not** caught by the `except`
- `finally` runs **always** — success, caught error, uncaught error, or `return`
- Use `finally` for cleanup: closing files, releasing connections, logging
- Full structure: `try` → `except` → `else` → `finally`
- `except` and `else` are both optional; `finally` is optional but very useful

---

## **Quick Reference**

```python
try:
    risky_op()          # The operation that might fail
except SomeError as e:
    handle_error(e)     # Runs ONLY if try raises SomeError
else:
    success_code()      # Runs ONLY if try raised NO exception
finally:
    cleanup()           # Runs ALWAYS

# When each part runs:
# try succeeds  →  try ✓  except ✗  else ✓  finally ✓
# try fails     →  try ✗  except ✓  else ✗  finally ✓
```